# Tutorial 8: A Deeper Look at Bayesian Inversion

In this tutorial, we'll explore the `LinearBayesianInversion` class in more detail. We will see how it combines information from a **prior** and a **likelihood** to produce a **posterior** distribution that represents our updated state of knowledge.

### The Bayesian Framework

The goal of Bayesian inversion is to compute the posterior probability distribution $p(u|d)$ using Bayes' theorem:
$$
p(u|d) = \frac{p(d|u) p(u)}{p(d)}
$$
* $p(u)$ is the **prior distribution**. This is a `GaussianMeasure` that encodes our knowledge about the model `u` *before* we see any data.
* $p(d|u)$ is the **likelihood**. This is determined by the forward problem and our data error statistics. It tells us how likely it is to observe data `d` given a particular model `u`.
* $p(u|d)$ is the **posterior distribution**. This is our updated `GaussianMeasure` for the model *after* observing the data. It combines the information from the prior and the likelihood.

### Is Bayesian Inversion "Better"?

Bayesian inversion is not inherently "better" than optimization methods; it is a different tool with different goals. Its main advantage is that it provides a full quantification of uncertainty. However, the result is only as good as the information you put in. A reasonable, well-justified **prior** is essential. A poor or incorrect prior can bias the result and lead to misleading conclusions.

In this tutorial, we will:
1.  Solve our circle inverse problem with a "good" prior.
2.  Solve the same problem again with a deliberately "wrong" prior.
3.  Compare the results to see the influence of the prior.

In [ ]:
# To run in colab, uncomment the line below to install pygeoinf. 
#%pip install pygeoinf --quiet

import numpy as np
import matplotlib.pyplot as plt
import pygeoinf as inf
from pygeoinf.symmetric_space.circle import Sobolev, CircleHelper

# For reproducibility
np.random.seed(42)

## 1. Setting up the Problem

First, let's set up the same inverse problem as before and generate our synthetic "true" model and data. The true model is generated from a prior with a characteristic length-scale of `0.1`.

In [ ]:
# --- Setup the forward problem ---
model_space = Sobolev.from_sobolev_parameters(2.0, 0.05)
n_data = 20
observation_points = model_space.random_points(n_data)
forward_operator = model_space.point_evaluation_operator(observation_points)
data_space = forward_operator.codomain
standard_deviation = 0.1
data_error_measure = inf.GaussianMeasure.from_standard_deviation(
    data_space, standard_deviation
)
forward_problem = inf.LinearForwardProblem(
    forward_operator, data_error_measure=data_error_measure
)

# --- Generate synthetic data from a "true" prior ---
# This prior assumes the function has a relatively short correlation length (scale=0.1)
true_prior = model_space.point_value_scaled_heat_kernel_gaussian_measure(
    0.1, 1.0
)
true_model, data = forward_problem.synthetic_model_and_data(true_prior)

print("Forward problem and synthetic data are ready.")

## 2. Inversion with a "Good" Prior

First, we perform the Bayesian inversion using a prior that is consistent with the process that generated the true data. This represents a situation where we have good prior knowledge about the system we are modeling.

In [ ]:
# This prior is the same as the one used to generate the data
good_prior = true_prior
solver = inf.CholeskySolver(galerkin=True)

# Set up and run the Bayesian inversion
bayesian_inversion_good = inf.LinearBayesianInversion(forward_problem, good_prior)
posterior_good = bayesian_inversion_good.model_posterior_measure(data, solver)
posterior_mean_good = posterior_good.expectation

# Estimate the pointwise uncertainty
low_rank_good = posterior_good.low_rank_approximation(10, method="variable", rtol = 1e-4)
variance_good = low_rank_good.sample_pointwise_variance(200)
std_dev_good = np.sqrt(variance_good)

print("Inversion with 'good' prior is complete.")


## 3. Inversion with a "Wrong" Prior

Now, let's see what happens if our prior information is misleading. We will define a new prior that assumes the function has a much **longer correlation length** (`scale=0.8`) than the true function. This represents a case where our initial assumptions about the system are incorrect.

In [ ]:
# This prior assumes the function is very smooth with a long correlation length
wrong_prior = model_space.point_value_scaled_heat_kernel_gaussian_measure(
    scale=0.8, amplitude=1.0
)

# Set up and run the Bayesian inversion with the wrong prior
bayesian_inversion_wrong = inf.LinearBayesianInversion(forward_problem, wrong_prior)
posterior_wrong = bayesian_inversion_wrong.model_posterior_measure(data, solver)
posterior_mean_wrong = posterior_wrong.expectation

# Estimate the pointwise uncertainty
low_rank_wrong = posterior_wrong.low_rank_approximation(10, method="variable", rtol=1e-4)
variance_wrong = low_rank_wrong.sample_pointwise_variance(200)
std_dev_wrong = np.sqrt(variance_wrong)

print("Inversion with 'wrong' prior is complete.")


## 4. Comparing the Results

Now, let's plot the results from both inversions side-by-side.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8), sharey=True)

# --- Plot 1: Result with the Good Prior ---
ax1.set_title("Result with a Good Prior (scale=0.1)", fontsize=16)
model_space.plot(true_model, fig=fig, ax=ax1, color="k", linestyle="--", label="True Model")
model_space.plot(posterior_mean_good, fig=fig, ax=ax1, color="b", label="Posterior Mean")
model_space.plot_error_bounds(
    posterior_mean_good, 2 * std_dev_good, fig=fig, ax=ax1, color="b", alpha=0.2
)
ax1.errorbar(observation_points, data, 2 * standard_deviation, fmt="ko", capsize=3, label="Data")
ax1.legend()
ax1.grid(True, linestyle=":")

# --- Plot 2: Result with the Wrong Prior ---
ax2.set_title("Result with a Wrong Prior (scale=0.8)", fontsize=16)
model_space.plot(true_model, fig=fig, ax=ax2, color="k", linestyle="--", label="True Model")
model_space.plot(posterior_mean_wrong, fig=fig, ax=ax2, color="r", label="Posterior Mean")
model_space.plot_error_bounds(
    posterior_mean_wrong, 2 * std_dev_wrong, fig=fig, ax=ax2, color="r", alpha=0.2
)
ax2.errorbar(observation_points, data, 2 * standard_deviation, fmt="ko", capsize=3, label="Data")
ax2.legend()
ax2.grid(True, linestyle=":")

plt.show()

### Conclusion

The results are clear:
* The inversion with the **good prior** produces a posterior that is close to the true model where the data is sufficient, and otherwise leads to realistically large uncertainties. 
* The inversion with the **wrong prior** produces a posterior mean that that is overly smooth though it does do a reasonable job of fitting the data. Crucially, however, the pointwise uncertainties are dramatically underestimated. 

This demonstrates the power and the main caveat of Bayesian methods: they provide a rigorous framework for combining prior knowledge with data, but the quality of the result depends directly on the quality of that prior knowledge.